In [8]:
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
import torch
import torch.nn.functional as F

In [9]:
from german_parser.util.const import CONSTS
from german_parser.util.dataloader import TigerDatasetGenerator
import dill as pickle
from string import punctuation

In [10]:
character_flag_generators = [
            lambda c: int(c.isupper()),
            lambda c: int(c.lower() in ["ä", "ö", "ü", "ß"]),
            lambda c: int(c.isdigit()),
            lambda c: int(c in punctuation)
        ]

In [11]:
dataset_generator = TigerDatasetGenerator(f"{CONSTS['data_dir']}/tiger/tiger_2.2_utf8.xml", (0.2, 0.1), character_flag_generators=character_flag_generators) # type:ignore

INFO:model:Parsing dataset from '/home/james/programming/ml/german/german_parser/util/../../data/tiger/tiger_2.2_utf8.xml'...


INFO:model:Parsed 50472 sentences.
INFO:model:Generating trees...
INFO:model:42261 (83.73%) trees generated.
INFO:model:Dataset split into 29582 training, 8452 dev, and 4227 test trees.


In [21]:
required_vars = (*dataset_generator.get_training_dataloader(batch_size=32, shuffle=True), dataset_generator.character_set, dataset_generator.character_flag_generators, dataset_generator.inverse_word_dict, dataset_generator.inverse_sym_set)

In [22]:
pickle.dump(required_vars, open("required_vars.pkl", "wb"))

In [23]:
train_dataloader, train_new_words, character_set, character_flag_generators, inverse_word_dict, inverse_sym_dict = pickle.load(open("required_vars.pkl", "rb"))